<img align="left" src = https://lsstdesc.org/assets/img/logo.png width=250 style="padding: 10px"> 
<b>Testing New DIA Kernel Bases</b> <br>
Contact author: Michael Wood-Vasey <br>
Last verified to run: 2023-06-05 <br>
LSST Science Pipelines version: Weekly 2023_21 <br>
Container Size: large <br>
Targeted learning level: intermediate <br>

Sets up a interactive stepping through of the tasks to do image subtraction to allow for easier modifications to StarSelector, Kernel bases, and Detection.

Note: This Notebook is written below the PipelineTask level.  Rather is uses individual Tasks directly and reads/writes output products to the butler.  This is pedagogically useful to understand how that works, and pratically helpful in working with the evolving `source_injection` package.  However, this structure is not scalable to larger runs (100+ images).  Such large-scale runs should be done as part of an integrated Task that can be connected and run through the large-scale cluster jobs submission.

1. [x] Find set of images that overlap
2. [x] Pick one as template, one as science
3. [x] Also deepCoadd.  Be able to use either.
4. [x] Run subtractions through Tasks
5. [x] Run detection and measurement through Task

In [ ]:
import os

from astropy.wcs import WCS

import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.display as afwDisplay
from lsst.afw.math import Warper, WarperConfig
from lsst.daf.butler import Butler
import lsst.geom as geom
from lsst.ip.diffim import AlardLuptonSubtractConfig, AlardLuptonSubtractTask
from lsst.ip.diffim import GetTemplateConfig, GetTemplateTask
from lsst.ip.diffim import DetectAndMeasureConfig, DetectAndMeasureTask
import lsst.sphgeom

Some things we may or may not want to import for overriding or inheriting:

In [ ]:
from lsst.pipe.tasks.makeWarp import MakeWarpConfig, MakeWarpTask
from lsst.ip.diffim import MakeKernelConfig, MakeKernelTask, PsfMatchConfig, PsfMatchConfigAL, PsfMatchConfigDF
from lsst.ip.diffim.subtractImages import _subtractImages
from lsst.ip.diffim.utils import evaluateMeanPsfFwhm, getPsfFwhm
from lsst.meas.algorithms import SourceDetectionTask, SubtractBackgroundTask
from lsst.meas.base import SingleFrameMeasurementTask
from lsst.pex.exceptions import InvalidParameterError

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')
plt.style.use('tableau-colorblind10')
%matplotlib inline

## Some helper utilities

In [ ]:
def show_image_on_wcs(calexp, figsize=(8, 8), ax=None, x=None, y=None,
                           pixel_extent=None, stamp_size=None,
                           vmin=-200, vmax=400,
                           marker="o", color="red", size=20):
    """
    Show an image with an RA, Dec grid overlaid.  Optionally add markers.
    
    Notes
    -----
    Specifying both pixel_extent and size is undefined.
    """
    if ax is None:
        fig = plt.figure(figsize=figsize)
        plt.subplot(projection=WCS(calexp.getWcs().getFitsMetadata()))
        ax = plt.gca()

    if stamp_size is not None and x is not None and y is not None:    
        half_stamp = stamp_size / 2
        # If x and y are of different types, then user should clarify what they wanted
        if np.isscalar(x):
            first_x = x
            first_y = y
        else:
            first_x = x[0]
            first_y = y[0]
            
        pixel_extent = (int(first_x - half_stamp), int(first_x + half_stamp),
                        int(first_y - half_stamp), int(first_y + half_stamp))
    if pixel_extent is None:
        pixel_extent = (0, calexp.width, 0, calexp.height)

    print(pixel_extent)
    # Image array is y, x.  
    # So we select from the image array in [Y_Begin:Y_End, X_Begin:X_End]
    # But then `extent` is (X_Begin, X_End, Y_Begin, Y_End)
    im = ax.imshow(calexp.image.array[pixel_extent[2]:pixel_extent[3],
                                      pixel_extent[0]:pixel_extent[1]],
                   cmap="gray", vmin=vmin, vmax=vmax,
                   extent=pixel_extent, origin="lower")
    ax.grid(color="white", ls="solid")
    ax.set_xlabel("Right Ascension")
    ax.set_ylabel("Declination")
    if x is not None and y is not None:
        ax.scatter(x, y, s=size, marker=marker, edgecolor=color, facecolor="none")
        ax.set_xlim(pixel_extent[0:2])
        ax.set_ylim(pixel_extent[2:4])

In [ ]:
def show_image_with_mask_plane(calexp, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.setMaskTransparency(80)
    display.setMaskPlaneColor('DETECTED', 'blue')
    display.mtv(calexp)
    plt.show()
    
    return display

In [ ]:
def htm_from_ra_dec_level(ra, dec, level):
    pixelization = lsst.sphgeom.HtmPixelization(level)
    htm_id = pixelization.index(
        lsst.sphgeom.UnitVector3d(
            lsst.sphgeom.LonLat.fromDegrees(ra, dec)
        )
    )
    return htm_id

In [ ]:
def get_dataset_refs_from_htm_list(dataset_type, htm_ids, level, aggregate="intersection"):
    hi = htm_ids[0]

    # dataset_refs is an iterator, but each query is only a few hundred results,
    #   so convert to a list for future convenience
    htm_kwargs = {}
    htm_kwargs[f"htm{level}"] = hi
    dataset_refs = list(butler.registry.queryDatasets(dataset_type, dataId={"band": band}, **htm_kwargs))
    dataset_refs = set(dataset_refs)
    
    for hi in htm_ids[1:]:
        htm_kwargs = {}
        htm_kwargs[f"htm{level}"] = hi
        dr = list(butler.registry.queryDatasets(dataset_type, dataId={"band": band}, **htm_kwargs))
        if aggregate == "intersection":
            dataset_refs = dataset_refs.intersection(set(dr))
        elif aggregate == "union":
            dataset_refs = dataset_refs.union(set(dr))
        else:
            print("Aggregation method '{aggregate}' not supported.")
            return
        
    return list(dataset_refs)

## Defining Dataset based on Site

We can run this on either DC2 or HSC by choosing appropriate RA, Dec

Currently (2023-05-26) DC2 is only available at the IDF and HSC is only available at the USDF,
so we split by site.

In [ ]:
SITE = "IDF"

survey_site = {"USDF": "HSC", "IDF": "DC2", "NERSC": "DC2"}
repo_site = {"USDF": "/repo/main", "IDF": "dp02", "NERSC": "/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo"}
collection_site = {"USDF": "HSC/runs/RC2/w_2023_15/DM-38691", "IDF": "2.2i/runs/DP0.2", "NERSC": "u/descdm/coadds_Y1_4638"}

ra_dec_survey = {"HSC": (150, 2.5), "DC2": (55, -30)}

In [ ]:
collection = collection_site[SITE]
repo_config = repo_site[SITE]

user = os.getenv("USER")
output_collection = f"u/{user}/test_dia"

In [ ]:
butler = Butler(repo_config, run=output_collection, collections=[output_collection, collection])

In [ ]:
# Do a spatial query for calexps using HTM levels following example in 04b_Intermediate_Butler_Queries.ipynb
ra, dec = ra_dec_survey[survey_site[SITE]]
band = "i"

In [ ]:
level = 20  # the resolution of the HTM grid
htm_id = htm_from_ra_dec_level(ra, dec, level)

In [ ]:
parent_level = htm_id // 10
htm_ids = [parent_level * 10 + i for i in [0, 1, 2, 3]]

In [ ]:
htm_ids

In [ ]:
dataset_refs = get_dataset_refs_from_htm_list("calexp", htm_ids, level)

# Sort by visitId to get a loose time order
ids_visit = [dr.dataId["visit"] for dr in dataset_refs]
dataset_refs = [dataset_refs[idx] for idx in np.argsort(ids_visit)]

print(dataset_refs)

In [ ]:
print(f"Found {len(list(dataset_refs))} calexps")

In [ ]:
visit_table = butler.get("visitTable")

We should find 140 calexps for DC2.  (RA, Dec) = (55, -30)  

We should find 44 calexps for HSC COSMOS.  (RA, Dec) = (150, +2.5)

# Build template for subtraction


Also provide a single image template based on calexp[0] in the list.

In [ ]:
single_image_template = butler.get("calexp", dataset_refs[0].dataId)

Run subtractions with calexp[1] as science.

In [ ]:
science_dr = dataset_refs[1]
science = butler.get("calexp", science_dr.dataId)

### Get a template from the deepCoadd
Here we get a template from the (tract, patch) deepCoadd reassembled to be continous across calexp.

In [ ]:
sky_map = butler.get("skyMap")

In [ ]:
get_template_task_config = GetTemplateConfig()
get_template_task = GetTemplateTask(config=get_template_task_config)

In [ ]:
bigger_level = 9
bigger_htm_id = htm_from_ra_dec_level(ra, dec, level=bigger_level)

coadd_exposure_refs = get_dataset_refs_from_htm_list("deepCoadd", [bigger_htm_id], level=bigger_level, aggregate="union")
coadd_exposure_deferred_dataset_handles = [butler.getDeferred(dr) for dr in coadd_exposure_refs]

In [ ]:
coadd_exposure_deferred_dataset_handles

Check that we're close to original RA, Dec

In [ ]:
science.getWcs().pixelToSky(science.getBBox().getCenter())

In [ ]:
inputs = {"coaddExposures" : coadd_exposure_deferred_dataset_handles,
          "bbox": science.getBBox(),
          "skyMap": sky_map,
          "wcs": science.getWcs(),
          "visitInfo": science.visitInfo,
         }

In [ ]:
results = get_template_task.getOverlappingExposures(inputs)
coadd_exposures = results.coaddExposures
data_ids = results.dataIds

In [ ]:
deep_coadd_template = get_template_task.run(coadd_exposures, inputs["bbox"], inputs["wcs"], data_ids)

In [ ]:
# Do I need to iterate or could I just del coadd_exposures with the same effect?
for ce in coadd_exposures:
    del ce

In [ ]:
show_image_on_wcs(science, vmin=-200, vmax=+400)

## Subtraction

In [ ]:
def warp(science, template):
    "Warp input template image to WCS and Bounding Box of the science image."
    warper_config = WarperConfig()
    warper = Warper.fromConfig(warper_config)

    science_wcs = science.getWcs()
    science_bbox = science.getBBox()
    
    warped_template = warper.warpExposure(science_wcs, template, destBBox=science_bbox)
    # Add PSF.  I think doing this directly without warping is wrong.
    # At least the x,y mapping should be updated
    warped_template.setPsf(template.getPsf())
    
    return warped_template


def subtract(science, template, source_catalog, task=None, config=None):
    # https://github.com/lsst/ip_diffim/blob/main/python/lsst/ip/diffim/subtractImages.py#L196
    if config is None and task is None:
        config = AlardLuptonSubtractConfig()
    if task is None:
        task = AlardLuptonSubtractTask(config=config)
    # Star Selection is done here:
    #   https://github.com/lsst/ip_diffim/blob/main/python/lsst/ip/diffim/subtractImages.py#L603

    warped_template = warp(science, template)
    
    subtraction = task.run(warped_template, science, source_catalog)
    
    return subtraction


def detect(science, subtraction):
    # Run detection on subtraction
    detect_and_measure_config = DetectAndMeasureConfig()
    detect_and_measure_task = DetectAndMeasureTask(config=detect_and_measure_config)

    detect_and_measure = detect_and_measure_task.run(science,
                                                     subtraction.matchedTemplate,
                                                     subtraction.difference)

    return detect_and_measure

## Provide a modified makeKernel

We can inherit from and then modify methods of the MakeKernelTask to test ideas for improvements.

The kernel used by AlardLuptonSubtractTask is a configurable option.

In [ ]:
# https://github.com/lsst/ip_diffim/blob/w.2023.07/python/lsst/ip/diffim/makeKernel.py#L45

class ModifiedMakeKernelConfig(MakeKernelConfig):
    """Stub inherited class to let room for future configuration passing"""
    # If you wanted to create a new config parameter to pass to the task:
    # foo = lsst.pex.config.ConfigChoiceField(doc="foo threshold", dtype=float, default=1.0)
    pass

In [ ]:
class ModifiedMakeKernelTask(MakeKernelTask):
    """Construct a kernel for PSF matching two exposures
    
    This Modified class is an example for showing to to create your own kernel-solving class.
    """
    ConfigClass = ModifiedMakeKernelConfig
    _DefaultName = "makeModifiedKernel"

    # This is the routine we might want to replace wtih our own ideas
    # about finding a good convolution kernel
    # Needs to return an lsst.afw.math.LinearCombinationKernel
    # Original
    #  https://github.com/lsst/ip_diffim/blob/main/python/lsst/ip/diffim/makeKernel.py#L108
    def run(self, template, science, kernelSources, preconvolved=False):
        """Solve for the kernel and background model that best match two
        Exposures evaluated at the given source locations.

        Parameters
        ----------
        template : `lsst.afw.image.Exposure`
            Exposure that will be convolved.
        science : `lsst.afw.image.Exposure`
            The exposure that will be matched.
        kernelSources : `list` of `dict`
            A list of dicts having a "source" and "footprint"
            field for the Sources deemed to be appropriate for Psf
            matching. Can be the output from ``selectKernelSources``.
        preconvolved : `bool`, optional
            Was the science image convolved with its own PSF?

        Returns
        -------
        results : `lsst.pipe.base.Struct`

            ``psfMatchingKernel`` : `lsst.afw.math.LinearCombinationKernel`
                Spatially varying Psf-matching kernel.
            ``backgroundModel``  : `lsst.afw.math.Function2D`
                Spatially varying background-matching function.
        """
        # Just debugging that we're really running this modified task
        self.log.info("Running Modified Make Kernel Task")

        kernelCellSet = self._buildCellSet(template.maskedImage, science.maskedImage, kernelSources)
        # Calling getPsfFwhm on template.psf fails on some rare occasions when
        # the template has no input exposures at the average position of the
        # stars. So we try getPsfFwhm first on template, and if that fails we
        # evaluate the PSF on a grid specified by fwhmExposure* fields.
        # To keep consistent definitions for PSF size on the template and
        # science images, we use the same method for both.
        try:
            templateFwhmPix = getPsfFwhm(template.psf)
            scienceFwhmPix = getPsfFwhm(science.psf)
        except InvalidParameterError:
            self.log.debug("Unable to evaluate PSF at the average position. "
                           "Evaluting PSF on a grid of points."
                           )
            templateFwhmPix = evaluateMeanPsfFwhm(template,
                                                  fwhmExposureBuffer=self.config.fwhmExposureBuffer,
                                                  fwhmExposureGrid=self.config.fwhmExposureGrid
                                                  )
            scienceFwhmPix = evaluateMeanPsfFwhm(science,
                                                 fwhmExposureBuffer=self.config.fwhmExposureBuffer,
                                                 fwhmExposureGrid=self.config.fwhmExposureGrid
                                                 )

        if preconvolved:
            scienceFwhmPix *= np.sqrt(2)
            
        ### THESE LINES ARE PROBABLY WHERE YOU WANT TO CHANGE: BEGIN ###
        basisList = self.makeKernelBasisList(templateFwhmPix, scienceFwhmPix,
                                             metadata=self.metadata)
        spatialSolution, psfMatchingKernel, backgroundModel = self._solve(kernelCellSet, basisList)
        ### END: THESE LINES ARE PROBABLY WHERE YOU WANT TO CHANGE
        
        return lsst.pipe.base.Struct(
            psfMatchingKernel=psfMatchingKernel,
            backgroundModel=backgroundModel,
        )

        

If we want to modify the run... method of the subtraction task itself, we would subclass AlardLuptonSubtractTask and modify the `run...` method.  Here we just pick "Modified" to be generic, but if one had a specific name, that would be good too.

In [ ]:
class ModifiedAlardLuptonSubtractConfig(AlardLuptonSubtractConfig):
    pass

class ModifiedAlardLuptonSubtractTask(AlardLuptonSubtractTask):
    ConfigClass = ModifiedAlardLuptonSubtractConfig
    _DefaultName = "modifiedAlardLuptonSubtract"
    
    # `run` calls `runConvolveTemplate` if the template PSF is better
    # Let's develop this example first before covering the `runConvolveScience` function.
    def runConvolveTemplate(self, template, science, selectSources):
        """Convolve the template image with a PSF-matching kernel and subtract
        from the science image.

        Parameters
        ----------
        template : `lsst.afw.image.ExposureF`
            Template exposure, warped to match the science exposure.
        science : `lsst.afw.image.ExposureF`
            Science exposure to subtract from the template.
        selectSources : `lsst.afw.table.SourceCatalog`
            Identified sources on the science exposure. This catalog is used to
            select sources in order to perform the AL PSF matching on stamp
            images around them.

        Returns
        -------
        results : `lsst.pipe.base.Struct`

            ``difference`` : `lsst.afw.image.ExposureF`
                Result of subtracting template and science.
            ``matchedTemplate`` : `lsst.afw.image.ExposureF`
                Warped and PSF-matched template exposure.
            ``backgroundModel`` : `lsst.afw.math.Function2D`
                Background model that was fit while solving for the PSF-matching kernel
            ``psfMatchingKernel`` : `lsst.afw.math.Kernel`
                Kernel used to PSF-match the template to the science image.
        """
        # Just some quick debugging to demonstrate that we are running the modifiedAlardLuptonSubtract
        self.log.info("Running Modified Subtraction Task")
        kernelSources = self.makeKernel.selectKernelSources(template, science,
                                                            candidateList=selectSources,
                                                            preconvolved=False)
        kernelResult = self.makeKernel.run(template, science, kernelSources,
                                           preconvolved=False)

        # https://github.com/lsst/ip_diffim/blob/main/python/lsst/ip/diffim/subtractImages.py#L564
        matchedTemplate = self._convolveExposure(template, kernelResult.psfMatchingKernel,
                                                 self.convolutionControl,
                                                 bbox=science.getBBox(),
                                                 psf=science.psf,
                                                 photoCalib=science.photoCalib)

        # The actual subtraction is very simple once we have the matchedTemplate and backgroundModel
        # It's so simple that it is a free function (that we imported above)
        difference = _subtractImages(science, matchedTemplate,
                                     backgroundModel=(kernelResult.backgroundModel
                                                      if self.config.doSubtractBackground else None))

        correctedExposure = self.finalize(template, science, difference,
                                          kernelResult.psfMatchingKernel,
                                          templateMatched=True)

        return lsst.pipe.base.Struct(difference=correctedExposure,
                                     matchedTemplate=matchedTemplate,
                                     matchedScience=science,
                                     backgroundModel=kernelResult.backgroundModel,
                                     psfMatchingKernel=kernelResult.psfMatchingKernel)
    


In [ ]:
subtract_config = AlardLuptonSubtractConfig()
subtract_config.makeKernel.retarget(ModifiedMakeKernelTask, ConfigClass=ModifiedMakeKernelConfig)
task = AlardLuptonSubtractTask(config=subtract_config)

In [ ]:
modified_subtract_config = ModifiedAlardLuptonSubtractConfig()
modified_task = ModifiedAlardLuptonSubtractTask(config=modified_subtract_config)

In [ ]:
# template = single_image_template
template = deep_coadd_template.template

In [ ]:
"""
Subtract template image from image referred to by data_id and run detection.
"""
science = butler.get("calexp", science_dr.dataId)
source_catalog = butler.get("src", dataId=science_dr.dataId)

In [ ]:
subtraction = subtract(science, template, source_catalog, task=task)

In [ ]:
modified_subtraction = subtract(science, template, source_catalog, task=modified_task)

I don't know why we don't get log messages from the modified task.  There's more to learn about logging.

In [ ]:
show_image_on_wcs(template, vmin=-1, vmax=+2)

In [ ]:
show_image_on_wcs(science)

In [ ]:
show_image_on_wcs(modified_subtraction.difference)

The negative regions above are saturated stars, as indicated by the masked-image view below where "green" is saturated.he negative regions above are saturated stars, as indicated by the masked-image view below where "green" is saturated.

Interpreting the above image plane correctly requires marking the saturated regions.  Stars brighter than ~17th mag will saturate in LSST images.  This means that the recording counts are not propotional to the flux, so the subtraction between two images of that field will not yield clean subtractions of the stars.  In general in one of the images the stars will be a little more saturated than the other and so have fewer proporational counts.  In this case for DC2, , it's the template image that has slightly more saturated stars (due to a higher sky brightness or a sharper PSF FWHM).T

In [ ]:
display = show_image_with_mask_plane(subtraction.difference)

In [ ]:
print("Mask plane bit definitions:\n", display.getMaskPlaneColor())
print("\nMask plane methods:\n")
help(display.setMaskPlaneColor)

In [ ]:
detection_catalog = detect(science, modified_subtraction)

## DIA Source Catalog

We can getting a better sense of the true performance of the image subtraction by looking at the catalog of detected and measured sources, the DIA Source Cstalog.

In [ ]:
dia_src = detection_catalog.diaSources.asAstropy()

In [ ]:
# Specific list.  But "base_PixelFlags_flag" should be set for any of these
full_list_pixelflags_indicating_bad_source = ["base_PixelFlags_flag_saturated",
"base_PixelFlags_flag_saturatedCenter",
"base_PixelFlags_flag_suspect",
"base_PixelFlags_flag_suspectCenter",
"base_PixelFlags_flag_offimage",
"base_PixelFlags_flag_edge",
"base_PixelFlags_flag_bad",]

There seem to be objects with some of the above flags set, but where "base_PixelFlags_flag" is not set.  Investigate.  This is a bug.

Apply flags that marker things that pipeline is indicating might be real transients.

In [ ]:
flags_indicating_bad_source = ["base_PixelFlags_flag_saturated",
                               "base_PixelFlags_flag_saturatedCenter",
                               "base_PixelFlags_flag_suspect",
                               "base_PixelFlags_flag_suspectCenter",
                               "base_PixelFlags_flag_offimage",
                               "base_PixelFlags_flag_edge",
                               "base_PixelFlags_flag_bad",
                               "base_SdssShape_flag",
                               "ip_diffim_DipoleFit_flag_classification",
                               "ip_diffim_DipoleFit_flag_classificationAttempted",
                               "base_GaussianFlux_flag",
                               "slot_Shape_flag",]

In [ ]:
bad = [dia_src[flag] for flag in flags_indicating_bad_source]
bad = np.any(np.vstack(bad), axis=0)

In [ ]:
good_dia_src = dia_src[~bad]

In [ ]:
print(f"Found {len(good_dia_src)} good DIA sources out of {len(dia_src)} DIA sources.")

In [ ]:
import re
shape_flags = [c for c in good_dia_src.columns if re.search("base.*Shape.*_.*flag", c)]
sdss_flags = [c for c in good_dia_src.columns if re.search("base.*Sdss.*_.*flag", c)]
slot_flags = [c for c in good_dia_src.columns if re.search("slot_.*flag", c)]

In [ ]:
x = good_dia_src["slot_Shape_x"]
y = good_dia_src["slot_Shape_y"]

In [ ]:
good_dia_src[["slot_PsfFlux_instFlux", "ip_diffim_forced_PsfFlux_instFlux", "ip_diffim_forced_PsfFlux_instFluxErr"]]

In [ ]:
if len(good_dia_src) >= 1:
    i = 0

    show_image_on_wcs(subtraction.matchedTemplate, x=x[i], y=y[i], stamp_size=100)

    show_image_on_wcs(subtraction.matchedScience, x=x[i], y=y[i], stamp_size=100)

    show_image_on_wcs(subtraction.difference, x=x[i], y=y[i], stamp_size=100)

    geom.Extent2I(100, 100)

    center = geom.SpherePoint(good_dia_src["coord_ra"][i], good_dia_src["coord_dec"][i], geom.radians)
    extent = geom.Extent2I(100, 100)
    cutout = subtraction.difference.getCutout(center, extent)

    show_image_with_mask_plane(cutout)

    good_dia_src[i][slot_flags]

    flags = [c for c in good_dia_src.columns if re.search("_flag", c)]

    good_dia_src[flags]